# 02 Generation Strategies

**本节目标**：

帮助读者快速掌握使用模型推理时的一些可配置参数

**备注**：

1. 本章节部分来自于Hugging Face的[Generation with LLMS](https://huggingface.co/docs/transformers/llm_tutorial)以及[Customize the generation strategy](https://huggingface.co/docs/transformers/generation_strategies)，如果有进一步的需求可以参阅源文档以获取更加准确的知识。

2. 由于本教程关注LLM，因此会忽略官方文档中不相关的部分，并且在本章节不会详细讲解原理。

## 长度控制
在hugging face的源文档中提到，模型的generation长度是可控的，并且在没有设置`GenerationConfig`时，默认返回20个token。

> If not specified in the GenerationConfig file, generate returns up to 20 tokens by default. We highly recommend manually setting max_new_tokens in your generate call to control the maximum number of new tokens it can return. Keep in mind LLMs (more precisely, decoder-only models) also return the input prompt as part of the output.
> 机翻：如果未在 GenerationConfig 文件中指定，generate 默认情况下最多返回 20 个token。我们强烈建议您在生成调用中手动设置 max_new_tokens 以控制它可以返回的新令牌的最大数量。请记住，LLM（更准确地说，仅解码器模型）也会将输入提示作为输出的一部分返回。

然而，在`Quick tour.ipynb`中，Tiny LLAMA生成了不止20个token，这是因为我们在下载`Tiny LLAMA`时，下载了`generation_config.json`这个文件

<img src='./src/TinyLLAMAconfig.png' alt='Tiny LLAMA config' width=1080>

在这个文件中，作者设置了如下的参数

`{
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 2048,
  "pad_token_id": 0,
  "transformers_version": "4.35.0"
}`

可以看到`max_length`为2048，因此不是默认的20，在接下来的部分中，我们将尝试通过两种方式控制generation的长度

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import tqdm as notebook_tqdm

model = AutoModelForCausalLM.from_pretrained(
    "../model/Tiny-LLAMA"
)

tokenizer = AutoTokenizer.from_pretrained("../model/Tiny-LLAMA", padding_side="left")

model_inputs = tokenizer(["please count number from 1 to 100, just like 1,2,3"], return_tensors="pt")

# Setting `max_new_tokens` allows you to control the maximum new token
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

# Setting `max_length` allows you to control the maximum length
generated_ids = model.generate(**model_inputs, max_length=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

please count number from 1 to 100, just like 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,2
please count number from 1 to 100, just like 1,2,3,4,5,6,7,8,9,10,11,12,13,14,1


可以看到，`max_length=50`相比`max_new_token=50`, 生成的generation更短，所以不难知道，max_length限制的是整个generation的长度，而max_new_token限制的是生成的新token数量，至于为什么50个token不能数到50，这是另一个问题了：）

## 生成策略

我们先看一组例子

In [11]:
# Set seed or reproducibility -- you don't need this unless you want full reproducibility
from transformers import set_seed
set_seed(42)

model_inputs = tokenizer(["I am a cat."], return_tensors="pt")

# LLM + greedy decoding = repetitive, boring output
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

generated_ids = model.generate(**model_inputs, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat.

2. "I am a dog. I am a dog. I am a dog."

3. "I am a bird. I am a bird. I am a bird."

4. "I am a fish.
I am a cat.

2. "I am a dog. I am a dog. I am a dog."

3. "I am a bird. I am a bird. I am a bird."

4. "I am a fish.


可以看到，在随机数种子固定的情况下，模型的生成完全一致，但在实际中，我们使用chatgpt之类的LLM时，却不会发生这样的问题。

<img src='./src//Chatgpt01.png' alt='chatgpt01' width=1080>

<img src='./src//Chatgpt02.png' alt='chatgpt02' width=1080>

这其中固然有随机数种子的原因，但同样受解码策略的影响。

> By default, and unless specified in the GenerationConfig file, generate selects the most likely token at each iteration (greedy decoding). Depending on your task, this may be undesirable; creative tasks like chatbots or writing an essay benefit from sampling. On the other hand, input-grounded tasks like audio transcription or translation benefit from greedy decoding. Enable sampling with do_sample=True, and you can learn more about this topic in this [blog post](https://huggingface.co/blog/how-to-generate).

> 机翻：默认情况下，除非在generationConfig文件中指定，否则生成每次迭代中最有可能的令牌（贪婪解码）。根据您的任务，这可能是不受欢迎的；诸如聊天机器人或撰写论文的创造性任务受益于抽样。另一方面，贪婪解码的输入接收任务（例如音频转录或翻译）受益。使用do_sample = true启用采样，您可以在此博客文章中了解有关此主题的更多信息。

在官方文档中，首先推荐使用`do_sample`参数，该参数可以让模型进行多项式采样，我们再次尝试,这次虽然是同样的随机数种子，但是生成出了两个完全不同的答案

In [15]:
# With sampling, the output becomes more creative!
generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])
print("------------------------")
generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat. But just so you know, I've been known to sneeze or scratch. 5. Kitty I am a mouse. I have a small body, but big ears! 6. Binky If you touch me
------------------------
I am a cat. I do not say I am a cat. I do not own you. I do not want to eat your lunch. I do not want to drink your wine. I do not live on your couch. I am a cat. I do


### 更多的生成控制
本部分来自于[生成策略的官方文档](https://huggingface.co/docs/transformers/generation_strategies)，并且该部分结合[GenerationMixin](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate)以及[GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig)进行简单讲解。

并且考虑到涉及太多源码会过于复杂，不适合初学者理解，因此这里尽可能的简化讲解，只讲解使用，详细的内容可以参考上面的链接详细阅读。

> 注：比较简单的`top k`,`top p`等策略在源码中被分类为logits_processor, 分开讲解

#### Greddy Search

贪婪搜索是最基础的采样方法，只要不设置其他的参数，例如`do_sample`以及我们之后会提到的`num_beams`,`penalty_alpha`等参数

In [17]:
# 贪婪搜索
model_inputs = tokenizer(["I am a cat."], return_tensors="pt")
generated_ids = model.generate(**model_inputs, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat.

2. "I am a dog. I am a dog. I am a dog."

3. "I am a bird. I am a bird. I am a bird."

4. "I am a fish.


#### 对比搜索
对比搜索常常用于减少重复输出，通常与`top_k`和`penalty_alpha`一起使用

In [18]:
generated_ids = model.generate(**model_inputs, penalty_alpha=0.6, top_k=4, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat.
My purrs are deafening,
I'm a cuddle monster,
And when I sleep, I snore.

But don't let my quirks fool you,
I have a heart


#### 多项式采样
多项式采样相比于贪婪搜索，允许整个词表根据概率被选择，而不是选择最大的，这提高了生成的多样性，如果使用多项式采样，需要`do_sample=True`和`num_beams=1`，该方法可以与`top k`和`top p`一起使用，如果使用的话就是在相应策略保留的token中进行多项式采样，而不是整个词表。

In [19]:
generated_ids = model.generate(**model_inputs, do_sample=True, top_k=4, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat. I am so happy and content.

I am a bird and I fly high in the sky, I am free and I am happy.

I have a tail and I wag it like I mean it!

I am a


#### Beam-Search
不同与贪婪搜索，Beam Search在每个时间步保留多个假设，并最终选择整个序列总体概率最高的假设，这样可以保留以较低概率初始标记开始，但最终概率较高的生成结果。使用该方法，需要至少将`num_beams`设置为大于1

In [20]:
generated_ids = model.generate(**model_inputs, num_beams=5, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat. I am a cat.


In [21]:
# Beam Search可以和多项式采样一起使用合并为具有随机性的Beam-search multinomial sampling方法
generated_ids = model.generate(**model_inputs, num_beams=5, do_sample=True, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat.

Scene 2:

(The cat is now sitting on a chair)

Cat: (whispering) I am a mouse.

Scene 3:

(The mouse is now sitting on a table


#### Diverse beam search decoding
该方法是beam search拓展方法可以允许更多样化的beam search生成结果，但需要更多的参数设置，具体设置相见代码

In [23]:
generated_ids = model.generate(**model_inputs, num_beams=5, num_beam_groups=5, max_new_tokens=50, diversity_penalty=1.0)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

I am a cat.
I am a cat. I am a cat. I am a cat.
I am a cat. I am a cat. I am a cat. I am a cat.
I am a cat. I am a cat. I am


### 可选参数
在该部分中，根据[GenerationConfig](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig)详细列出generate可以选择填的参数，以及大概的作用（这里选择一些常用的进行翻译，而不是全部）
#### 控制输出长度的参数
max_length (整型，可选，默认为20) - 生成的token的最大长度。等于输入提示的长度加上max_new_tokens的值。如果同时设置了max_new_tokens，则其效果会被覆盖。

例：如果输入提示为5个token，max_length设置为20，则生成的文本最长为20个token。

max_new_tokens (整型，可选) - 生成的最大token数，忽略提示中的token数量。

例：无论输入提示多长，如果max_new_tokens设置为15，那么将只生成最多15个新的token。

min_length (整型，可选，默认为0) - 要生成的序列的最小长度。等于输入提示的长度加上min_new_tokens的值。如果同时设置了min_new_tokens，则其效果会被覆盖。

例：如果输入提示为5个token，min_length设置为10，则生成的文本至少为10个token。

min_new_tokens (整型，可选) - 生成的最小token数，忽略提示中的token数量。

例：无论输入提示多长，如果min_new_tokens设置为5，那么至少会生成5个新的token。

early_stopping (布尔值或字符串，可选，默认为False) - 控制基于beam的方法（如beam-search）的停止条件。接受以下值：True，当有num_beams个完整候选项时停止生成；False，应用启发式方法，当找到更好的候选项变得非常不可能时停止生成；"never"，只有当没有更好的候选项时beam search程序才停止（标准的beam search算法）。

例：在使用beam search时，early_stopping设置为True可以在找到足够数量的候选项后快速停止搜索。

max_time (浮点型，可选) - 允许计算运行的最大时间（秒）。即使分配的时间已经过去，生成仍将完成当前的计算。

#### 控制所用生成策略的参数
do_sample (布尔值，可选，默认为False) - 是否使用采样；否则使用贪婪解码。

例：开启do_sample可以让生成的文本更加多样化，而不总是选择最可能的token。

num_beams (整型，可选，默认为1) - beam search的beam数量。1表示不使用beam search。

例：增加num_beams的值可以提高生成文本的质量，但会增加计算成本。

num_beam_groups (整型，可选，默认为1) - 将num_beams分成若干组，以保证不同beam组之间的多样性。

例：如果num_beams为4，num_beam_groups为2，那么会有2组beam，每组2个。

penalty_alpha (浮点型，可选) - 平衡模型置信度和对比搜索解码中的退化惩罚的值。

use_cache (布尔值，可选，默认为True) - 模型是否应该使用过去的最后一个key/values注意力（如果适用于模型）来加速解码。

#### 操纵模型输出logits的参数
temperature (浮点型，可选，默认为1.0) - 用于调节下一个token概率的值。

例：降低temperature可以使生成的文本更加确定性，提高则增加随机性。

top_k (整型，可选，默认为50) - 在top-k过滤中保留的最高概率词汇token的数量。

例：如果top_k设置为10，则每次生成token时只从概率最高的10个中选择。

top_p (浮点型，可选，默认为1.0) - 如果设置为小于1的浮点数，则仅保留总概率之和至少为top_p的最可能的token集合以用于生成。

例：top_p设为0.9，则每次生成时只考虑累积概率达到0.9的那部分token。

diversity_penalty (浮点型，可选，默认为0.0) - 如果某个beam在特定时间生成了与其他组的任何beam相同的token，则从该beam的分数中扣除此值。注意，只有启用了组beam搜索时，diversity_penalty才有效。

例：设置diversity_penalty可以避免不同beam生成重复的内容。

repetition_penalty (浮点型，可选，默认为1.0) - 重复惩罚的参数。1.0意味着没有惩罚。

例：提高repetition_penalty可以减少生成内容中的重复。

length_penalty (浮点型，可选，默认为1.0) - 在基于beam的生成中使用的长度惩罚。它作为序列长度的指数应用，用于分配序列的分数。由于分数是序列的对数似然（即负值），因此length_penalty > 0.0倾向于产生更长的序列，而length_penalty < 0.0则鼓励更短的序列。

例：提高length_penalty可以促使模型生成更长的文本。

no_repeat_ngram_size (整型，可选，默认为0) - 如果设置为大于0的整数，则所有该大小的ngram只能出现一次。

例：设置no_repeat_ngram_size为2可以防止任何两个词的组合在文本中重复出现。

定义generate输出变量的参数
num_return_sequences (整型，可选，默认为1) - 每个批次元素独立计算返回的序列数。

例：提高num_return_sequences可以获得更多不同的生成结果。

output_attentions (布尔值，可选，默认为False) - 是否返回所有注意力层的注意力张量。

#### 在生成时可以使用的特殊token
pad_token_id (整型，可选) - 填充token的id。

bos_token_id (整型，可选) - 序列开始token的id。

eos_token_id (整型或整型列表，可选) - 序列结束token的id。可选地，使用列表设置多个结束序列token。
